# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [3]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama

In [2]:
# Use local LLAMA3.2 instance
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"


In [27]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [4]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
collaboratory = Website("https://www.valpo.edu/collaboratory")
collaboratory.links

['#main-content',
 '#',
 'https://www.valpo.edu',
 'https://www.valpo.edu/academics3/',
 'https://www.valpo.edu/admission-aid/',
 'https://www.valpo.edu/admission-aid/undergraduate-2/tuition-and-aid/',
 'https://www.valpo.edu/life-at-valpo/',
 'https://www.valpo.edu/apply-to-valpo/',
 'https://www.valpo.edu/request-info/',
 'https://www.valpo.edu/visit-2/',
 'https://www.valpo.edu/academics3/',
 'https://www.valpo.edu/admission-aid/',
 'https://www.valpo.edu/admission-aid/undergraduate-2/tuition-and-aid/',
 'https://www.valpo.edu/life-at-valpo/',
 '#',
 '/about',
 'https://www.valpoathletics.com/',
 'https://www.valpo.edu/stories/',
 'https://valpo.edu/calendar',
 'https://alumni.valpo.edu/',
 'https://www.valpo.edu/resource-hub/',
 'https://www.valpo.edu/gateway/faculty-staff/',
 'https://www.valpo.edu/campus-health/',
 'https://www.valpo.edu/apply-to-valpo/',
 'https://www.valpo.edu/request-info/',
 'https://www.valpo.edu/visit-2/',
 'https://www.valpo.edu/collaboratory/about-us/',
 

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [8]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
print(get_links_user_prompt(collaboratory))

Here is the list of links on the website of https://www.valpo.edu/collaboratory - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
#main-content
#
https://www.valpo.edu
https://www.valpo.edu/academics3/
https://www.valpo.edu/admission-aid/
https://www.valpo.edu/admission-aid/undergraduate-2/tuition-and-aid/
https://www.valpo.edu/life-at-valpo/
https://www.valpo.edu/apply-to-valpo/
https://www.valpo.edu/request-info/
https://www.valpo.edu/visit-2/
https://www.valpo.edu/academics3/
https://www.valpo.edu/admission-aid/
https://www.valpo.edu/admission-aid/undergraduate-2/tuition-and-aid/
https://www.valpo.edu/life-at-valpo/
#
/about
https://www.valpoathletics.com/
https://www.valpo.edu/stories/
https://valpo.edu/calendar
https://alumni.valpo.edu/
https://www.valpo.edu/resource-hub/
https://www.valpo.edu/gateway/fac

In [24]:
## For local LLAMA instance
def get_links(url):
    website = Website(url)
    response = ollama.chat(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
        ]
    )
    result = response['message']['content']
    # Parse the response
    if response and "content" in response:
        result = response["content"]
        return json.loads(result)
    else:
        raise ValueError("Invalid response from Ollama")

In [28]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [32]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

collaboratory = Website("https://www.valpo.edu/collaboratory")
collaboratory.links

['#main-content',
 '#',
 'https://www.valpo.edu',
 'https://www.valpo.edu/academics3/',
 'https://www.valpo.edu/admission-aid/',
 'https://www.valpo.edu/admission-aid/undergraduate-2/tuition-and-aid/',
 'https://www.valpo.edu/life-at-valpo/',
 'https://www.valpo.edu/apply-to-valpo/',
 'https://www.valpo.edu/request-info/',
 'https://www.valpo.edu/visit-2/',
 'https://www.valpo.edu/academics3/',
 'https://www.valpo.edu/admission-aid/',
 'https://www.valpo.edu/admission-aid/undergraduate-2/tuition-and-aid/',
 'https://www.valpo.edu/life-at-valpo/',
 '#',
 '/about',
 'https://www.valpoathletics.com/',
 'https://www.valpo.edu/stories/',
 'https://valpo.edu/calendar',
 'https://alumni.valpo.edu/',
 'https://www.valpo.edu/resource-hub/',
 'https://www.valpo.edu/gateway/faculty-staff/',
 'https://www.valpo.edu/campus-health/',
 'https://www.valpo.edu/apply-to-valpo/',
 'https://www.valpo.edu/request-info/',
 'https://www.valpo.edu/visit-2/',
 'https://www.valpo.edu/collaboratory/about-us/',
 

In [30]:
get_links("https://www.valpo.edu/collaboratory")

{'links': [{'type': 'about page',
   'url': 'https://www.valpo.edu/collaboratory/about-us/'},
  {'type': 'company page', 'url': 'https://www.valpo.edu/collaboratory'},
  {'type': 'careers page', 'url': 'https://jobs.silkroad.com/Valpo/Careers'},
  {'type': 'admissions page', 'url': 'https://www.valpo.edu/admission-aid/'},
  {'type': 'tuition and aid page',
   'url': 'https://www.valpo.edu/admission-aid/undergraduate-2/tuition-and-aid/'},
  {'type': 'life at Valpo page',
   'url': 'https://www.valpo.edu/life-at-valpo/'},
  {'type': 'events calendar page', 'url': 'https://valpo.edu/calendar'},
  {'type': 'news page', 'url': 'https://www.valpo.edu/news/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [33]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [34]:
print(get_all_details("https://www.valpo.edu/collaboratory"))

Found links: {'links': [{'type': 'about page', 'url': 'https://www.valpo.edu/collaboratory/about-us/'}, {'type': 'careers page', 'url': 'https://jobs.silkroad.com/Valpo/Careers'}, {'type': 'main website', 'url': 'https://www.valpo.edu'}, {'type': 'admissions page', 'url': 'https://www.valpo.edu/admission-aid/'}, {'type': 'academics page', 'url': 'https://www.valpo.edu/academics3/'}]}
Landing page:
Webpage Title:
- The Collaboratory
Webpage Contents:
Skip to content
Search
close
Academics
Admission
Scholarships & Aid
Life at Valpo
Apply
Request Info
Visit
Academics
Admission
Scholarships & Aid
Life at Valpo
MORE
About
Athletics
News & Stories
Events
Alumni
Current Students
Faculty & Staff
Campus Health Info
Apply
Request Info
Visit
The Collaboratory
About Us
Get Involved
BioME Sector Alliance
For Faculty
How Faculty Can Get Involved
For Other Universities
Contact
Home
About Us
Get Involved
For Faculty
For Other Universities
Contact
Unlock the Future of Innovation with the Valpo Collabor

In [35]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [36]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:1_000] # Truncate if more than 1,000 characters
    return user_prompt

In [ ]:
get_brochure_user_prompt("Valparaiso University Collaboratory", "https://www.valpo.edu/collaboratory")

In [37]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [38]:
create_brochure("Valparaiso University Collaboratory", "https://www.valpo.edu/collaboratory")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.valpo.edu/collaboratory/about-us/'}, {'type': 'main company page', 'url': 'https://www.valpo.edu/collaboratory'}, {'type': 'careers page', 'url': 'https://jobs.silkroad.com/Valpo/Careers'}, {'type': 'admissions page', 'url': 'https://www.valpo.edu/admission-aid/'}, {'type': 'academic programs', 'url': 'https://www.valpo.edu/academics3/'}]}


```markdown
# Valparaiso University Collaboratory

## Unlock the Future of Innovation

Welcome to the Valparaiso University Collaboratory, where academia meets industry to foster innovative solutions to the world’s toughest challenges. Collaborate with talented students and faculty members to elevate your projects and drive meaningful results.

### About Us

The Collaboratory engages in a multitude of sectors, facilitating partnerships that bridge the gap between theoretical research and practical application. Our mission is to leverage the collective genius of Valparaiso University’s brightest minds in collaboration with diverse industry leaders.

### Our Culture

At Valparaiso University, we pride ourselves on a culture of inquiry, collaboration, and mutual respect. The Collaboratory embodies these values by creating an inclusive environment that encourages creativity and innovation. Our community is driven by a commitment to social responsibility, academic excellence, and professional integrity.

### Who We Serve

The Collaboratory caters to a diverse clientele, ranging from small startups to large enterprises. We partner with various industries through initiatives like the BioME Sector Alliance, which focuses on bioscience and medical engineering challenges. By offering tailored solutions and fresh perspectives, we aim to help organizations innovate and succeed.

### Get Involved

Whether you’re a business looking to solve pressing challenges, a faculty member seeking to get involved, or a university from another region aiming to collaborate, the Collaboratory is open for partnership. We invite you to join us on this innovation journey.

### Careers and Opportunities

The Collaboratory is not just about helping industries; it also helps shape future leaders. Students at Valparaiso University gain invaluable hands-on experience through involvement in projects that address real-world problems. As part of our culture, we prioritize growth, learning, and professional development, making it an ideal environment for career advancement.

### Contact Us

For more information on how you can partner with us, please reach out through our [Contact Page](#).

Unlock your organization’s potential through collaboration with Valparaiso University’s Collaboratory—where innovation meets industry to create solutions for tomorrow.
```


## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [39]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [40]:
stream_brochure("Valparaiso University Collaboratory", "https://www.valpo.edu/collaboratory")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.valpo.edu/collaboratory/about-us/'}, {'type': 'company page', 'url': 'https://www.valpo.edu/collaboratory'}, {'type': 'careers page', 'url': 'https://jobs.silkroad.com/Valpo/Careers'}, {'type': 'admissions page', 'url': 'https://www.valpo.edu/admission-aid/'}, {'type': 'tuition and aid page', 'url': 'https://www.valpo.edu/admission-aid/undergraduate-2/tuition-and-aid/'}]}


# Valparaiso University Collaboratory Brochure

## Unlock the Future of Innovation

Welcome to the **Valparaiso University Collaboratory**, where we foster collaboration between academia and industry to pave the way for innovative solutions that address the toughest challenges in various sectors. Our mission is to partner with industry leaders, researchers, and students to unlock groundbreaking advancements that can reshape our future.

### About Us

The Collaboratory serves as a hub for research and innovation, connecting Valparaiso University’s brightest minds with businesses and organizations. We focus on multidisciplinary approaches that harness diverse expertise to drive progress and impact.

### Our Vision

At the Collaboratory, we believe in the power of collaboration. By pooling our resources and knowledge, we aim to create transformative outcomes that resonate in the academic world and beyond. Our research initiatives are designed not just to satisfy academic curiosity, but to deliver real-world solutions.

### Get Involved

Whether you are an industry leader looking to solve complex issues or an academic interested in pushing the boundaries of research, we invite you to join us in our mission.

- **For Industry:** Partner with our talented researchers and students to tackle specific challenges in your sector.
- **For Faculty:** Engage in interdisciplinary research projects that challenge conventional thinking.
- **For Other Universities:** Explore collaborative opportunities that expand the impact of your research initiatives.

### Customer Engagement

We cater to a diverse range of customers, including:
- **Local Businesses:** Leveraging campus resources for tailored research.
- **Nonprofits:** Collaborating on social impact projects to address community needs.
- **Corporate Partners:** Working on innovative technologies and solutions that align with industry goals.

### Company Culture

Valparaiso University fosters a culture of innovation, inclusivity, and continuous learning. We are dedicated to providing a supportive environment where all team members—students, faculty, and collaborators—can thrive. We encourage creativity, open dialogue, and a shared commitment to excellence.

### Careers and Opportunities

Joining the Valparaiso University Collaboratory opens doors to a myriad of professional opportunities. We seek individuals who are passionate about research, innovation, and collaboration. Explore career options that allow you to work at the cutting edge of industry and academia, making significant contributions to your field.

### Contact Us

Ready to embark on a journey of discovery and innovation? Reach out to us to learn more about partnership opportunities, research initiatives, or career advancements.

- **Website:** [Valparaiso University Collaboratory](#)
- **Email:** [contact@valpo.edu](mailto:contact@valpo.edu)
- **Phone:** 1-800-123-4567

---

Join us in unlocking the future of innovation with the Valparaiso University Collaboratory—where collaboration meets cutting-edge research.

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>